In [100]:
# Import the libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import keras
from keras.layers import Input, Dense, Lambda, Dropout
from keras.models import Model, Sequential
from keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix, cohen_kappa_score

In [101]:
# Load and preprocess the dataset
data = pd.read_csv('EEG_Eye_State.csv')
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

from imblearn.over_sampling import SMOTE
su = SMOTE(random_state=42)
X, y= su.fit_resample(X, y)

# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(data.drop('Class', axis=1))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [102]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14980 entries, 0 to 14979
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      14980 non-null  float64
 1   V2      14980 non-null  float64
 2   V3      14980 non-null  float64
 3   V4      14980 non-null  float64
 4   V5      14980 non-null  float64
 5   V6      14980 non-null  float64
 6   V7      14980 non-null  float64
 7   V8      14980 non-null  float64
 8   V9      14980 non-null  float64
 9   V10     14980 non-null  float64
 10  V11     14980 non-null  float64
 11  V12     14980 non-null  float64
 12  V13     14980 non-null  float64
 13  V14     14980 non-null  float64
 14  Class   14980 non-null  int64  
dtypes: float64(14), int64(1)
memory usage: 1.7 MB


In [103]:
data.Class.value_counts()

1    8257
0    6723
Name: Class, dtype: int64

In [104]:
y.value_counts()

1    8257
0    8257
Name: Class, dtype: int64

In [105]:
# Define the VAE architecture
original_dim = X_train.shape[1]
latent_dim = 2
intermediate_dim = 128

In [106]:
# Encoder network
inputs = Input(shape=(original_dim,))
h = Dense(intermediate_dim, activation='relu')(inputs)
h = Dense(intermediate_dim, activation='relu')(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

In [107]:
# Reparameterization trick
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0., stddev=1.)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

In [108]:
# Decoder network
h_decoded = Dense(intermediate_dim, activation='relu')(z)
h_decoded = Dense(intermediate_dim, activation='relu')(h_decoded)
x_decoded = Dense(original_dim, activation='linear')(h_decoded)

In [109]:
# Define the VAE model
vae = Model(inputs, x_decoded)
reconst_loss = original_dim * keras.losses.binary_crossentropy(inputs, x_decoded) 
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconst_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='rmsprop')

In [110]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', patience=10, mode='min', restore_best_weights=True)
# Train the VAE
vae.fit(X_train, X_train,
        shuffle=True,
        epochs=50,
        batch_size=128,
        validation_data=(X_test, X_test),
        callbacks=[es])

Epoch 1/50
104/104 [==============================] - 2s 9ms/step - loss: 5.1456 - val_loss: 206863.3281
Epoch 2/50
104/104 [==============================] - 1s 8ms/step - loss: 3.1755 - val_loss: 2874503.0000
Epoch 3/50
104/104 [==============================] - 1s 7ms/step - loss: 2.7119 - val_loss: 45019.7695
Epoch 4/50
104/104 [==============================] - 1s 7ms/step - loss: 2.6377 - val_loss: 129443.2031
Epoch 5/50
104/104 [==============================] - 1s 7ms/step - loss: 2.6353 - val_loss: 17689.9551
Epoch 6/50
104/104 [==============================] - 1s 7ms/step - loss: 2.5986 - val_loss: 74098984.0000
Epoch 7/50
104/104 [==============================] - 1s 7ms/step - loss: 2.5802 - val_loss: 362333.3125
Epoch 8/50
104/104 [==============================] - 1s 7ms/step - loss: 2.5807 - val_loss: 851639.6250
Epoch 9/50
104/104 [==============================] - 1s 7ms/step - loss: 2.5645 - val_loss: 440721472.0000
Epoch 10/50
104/104 [==============================

In [111]:
# vae.summary()
# len(vae.layers)

In [112]:
# Use the encoder to get latent variables for each data point
# encoder = Model(inputs, z_mean)
encoder = Sequential()
for i in range((len(vae.layers)-15)//2):
	encoder.add(vae.layers[i])
	
X_train_encoded = encoder.predict(X_train)
X_test_encoded = encoder.predict(X_test)

104/104 [==============================] - 0s 1ms/step


In [113]:
X_train_encoded.shape

(13211, 2)

In [114]:
# # Define a classifier on top of the encoded data
# clf_inputs = Input(shape=(latent_dim,))
# clf_h = Dense(32, activation='relu')(clf_inputs)
# clf_h = Dense(16, activation='relu')(clf_h)
# clf_h = Dense(8, activation='relu')(clf_h)
# clf_outputs = Dense(1, activation='sigmoid')(clf_h)
# clf = Model(clf_inputs, clf_outputs)

from keras.constraints import MaxNorm
clf = Sequential()
clf.add(Dense(32, input_shape=(latent_dim,), activation='relu', kernel_constraint=MaxNorm(3)))
clf.add(Dropout(0.2))
clf.add(Dense(8, activation='relu', kernel_constraint=MaxNorm(3)))
clf.add(Dropout(0.2))
clf.add(Dense(1, activation='sigmoid'))

In [115]:
# Train the classifier on the encoded data
from keras.optimizers import Adam

clf.compile(optimizer=Adam(learning_rate=0.01), loss='mse', metrics=['accuracy'])
clf.fit(X_train_encoded, 
        y_train, epochs=50, 
        batch_size=128, 
        validation_split=0.25, 
        verbose=2,
        callbacks=[es])

Epoch 1/50
78/78 - 1s - loss: 0.2501 - accuracy: 0.5000 - val_loss: 0.2500 - val_accuracy: 0.4989 - 832ms/epoch - 11ms/step
Epoch 2/50
78/78 - 0s - loss: 0.2502 - accuracy: 0.4965 - val_loss: 0.2500 - val_accuracy: 0.5011 - 330ms/epoch - 4ms/step
Epoch 3/50
78/78 - 0s - loss: 0.2501 - accuracy: 0.5007 - val_loss: 0.2500 - val_accuracy: 0.5011 - 322ms/epoch - 4ms/step
Epoch 4/50
78/78 - 0s - loss: 0.2501 - accuracy: 0.4990 - val_loss: 0.2500 - val_accuracy: 0.5011 - 312ms/epoch - 4ms/step
Epoch 5/50
78/78 - 0s - loss: 0.2500 - accuracy: 0.4979 - val_loss: 0.2500 - val_accuracy: 0.5011 - 322ms/epoch - 4ms/step
Epoch 6/50
78/78 - 0s - loss: 0.2500 - accuracy: 0.4994 - val_loss: 0.2500 - val_accuracy: 0.5011 - 321ms/epoch - 4ms/step
Epoch 7/50
78/78 - 0s - loss: 0.2501 - accuracy: 0.5056 - val_loss: 0.2501 - val_accuracy: 0.4989 - 313ms/epoch - 4ms/step
Epoch 8/50
78/78 - 0s - loss: 0.2501 - accuracy: 0.5001 - val_loss: 0.2500 - val_accuracy: 0.4989 - 319ms/epoch - 4ms/step
Epoch 9/50
78/7

In [116]:
# Evaluate the performance of the model on the test set
test_loss, test_acc = clf.evaluate(X_test_encoded, y_test, verbose=2)
print('Test accuracy:', test_acc)

104/104 - 0s - loss: 0.2501 - accuracy: 0.5017 - 200ms/epoch - 2ms/step
Test accuracy: 0.5016651749610901


In [117]:
# Use the trained model to make predictions on new data
predictions = clf.predict(X_test_encoded)

# Convert the predictions to binary values
predictions = np.round(predictions)

104/104 [==============================] - 0s 1ms/step


In [118]:
# Print the classification report and confusion matrix
print(classification_report(y_test, predictions))
print("Kapaa:", cohen_kappa_score(y_test, predictions))
print("\nConfusion_matrix: \n", confusion_matrix(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1646
           1       0.50      1.00      0.67      1657

    accuracy                           0.50      3303
   macro avg       0.25      0.50      0.33      3303
weighted avg       0.25      0.50      0.34      3303

Kapaa: 0.0

Confusion_matrix: 
 [[   0 1646]
 [   0 1657]]


c:\Users\kj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\kj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\kj\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier